In [28]:
#1.피처엔지니어링 추가한것넣기
#2.베이지안최적화를 통해 lightgbm

In [61]:
#베이스라인만들어보자
import numpy as np
import pandas as pd
#train=pd.read_csv("/kaggle/input/playground-series-s4e1/train.csv")
#test=pd.read_csv("/kaggle/input/playground-series-s4e1/test.csv")
#submission=pd.read_csv("/kaggle/input/playground-series-s4e1/sample_submission.csv")
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
submission=pd.read_csv("sample_submission.csv")
#ROC이용
print(train.shape,test.shape,submission.shape)

#Exited대신 target사용하겠습니다
train.rename(columns={'Exited':'target'}, inplace=True)
test.rename(columns={'Exited':'target'}, inplace=True)

(165034, 14) (110023, 13) (110023, 2)


In [30]:
#데이터 합치기
all_data=pd.concat([train,test],ignore_index=True)
all_data=all_data.drop('target',axis=1)#타깃값제거
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275057 entries, 0 to 275056
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               275057 non-null  int64  
 1   CustomerId       275057 non-null  int64  
 2   Surname          275057 non-null  object 
 3   CreditScore      275057 non-null  int64  
 4   Geography        275057 non-null  object 
 5   Gender           275057 non-null  object 
 6   Age              275057 non-null  float64
 7   Tenure           275057 non-null  int64  
 8   Balance          275057 non-null  float64
 9   NumOfProducts    275057 non-null  int64  
 10  HasCrCard        275057 non-null  float64
 11  IsActiveMember   275057 non-null  float64
 12  EstimatedSalary  275057 non-null  float64
dtypes: float64(5), int64(5), object(3)
memory usage: 27.3+ MB


In [67]:
#feature engineering
#간단한 전처리(feature engineering)
df=all_data

#ordinal encoding은 label도 순서지만 내가 원하는 순서대로 인코딩하고 싶을땐 map함수이용해서하거나 아니면 알파벳은 ordinal encoder쓰면
#편함 그냥 라벨은 순서가 고려되긴하는데 막 넣을수있기에 조심해서보고해야함.
#sklearn의 모델이나 xgboost는 반드시 레이블이나 원핫 해줘야함 
#lightgbm ,catboost ,tree기반 ,statsmodel은 알아서 해줌(더미변수화든 원핫이든 레이블이든)

#목적변수는 int형으로 가만히 두기
#Xgboost를 R에서 사용할때는 상관 없지만 Python에서 사용할때는 
#범주형 변수들을 (Object나 Category로 되어 있는 변수) 숫자형 변수로 바꿔줘야 한다.
#그래서 0과1로되어있는 이진변수(int)까지는 숫자형으로 바꿀필요없는듯(xgboost도 마찬가지)
#귀찮은거 많이 없애주고 속도도빠르니 lightgbm쓰나벼?

#feature engineering 고려해야할 부분!!!!!!!!!!!!!
#1.데이터 하나당 결측값 개수를 파생피처로 추가할수 있다.(우리는 아쉽게도 결측값 안존재)
#2.명목형 피처의 고유값 별 개수를 만들어볼 수 있다.



#중복값이면 1아니면 0인 feature를 df에 추가한다.
df['duple'] = df['CustomerId'].duplicated(keep=False).astype('int')





#3.Geography와 Gender는 object지만 나중을 위해 숫자로 변환
#3개니까 Geography 원핫 인코딩 진행(순서를 고려해야할경우는 label인코딩을 해야함)
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, drop='first')
df_encoded = pd.DataFrame(encoder.fit_transform(df[['Geography']]), columns=encoder.get_feature_names_out(['Geography']))
# 기존 데이터프레임에서 'Geography' 열 제거
df = df.drop(['Geography'], axis=1)
# 인코딩된 결과를 기존 데이터프레임에 합치기
df = pd.concat([df, df_encoded], axis=1)


#2개니까 gender는 원핫인코딩필요 없음 레이블 인코디진행 0과1이잖아
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])
df['Gender'] = df['Gender'].astype('int')





#5.Surname과 관련된 onehot인코딩
#6.CustomerId 와 관련된 onehot인코딩
#위의 두개를 한번에 적용한다.한번에 적용해지면 너무 커서 메모리할당안됨....어쩌지?
#cat_features=['Surname','CustomerId']
#from sklearn.preprocessing import OneHotEncoder
#encoder = OneHotEncoder(sparse_output=False, drop='first')
#encoded_cat_matrix=encoder.fit_transform(df[cat_features])
#encoded_cat_matrix


#7.id없애기
df = df.drop(['id'], axis=1)

#8.Balance 0인지 아닌지에 대한 feature생성
df['Balance2'] = (df['Balance'] == 0).astype('int')

#9 CreditScore 850인지 아닌지에 대한 feature 생성
df['CreditScore2'] = (df['CreditScore'] == 0).astype('int')

#10 Balance가 20000보다 크냐 아니냐에 따라 feature생성
df['Balance3'] = (df['Balance'] > 200000).astype(int)


#6.따로따로 한다음에 합칠때 csr형식으로 합쳐보자
#따로따로해도 크기 때문에 둘중 선택해야함(해봄)
#CustomerId는 혼자해도 커서 일단 포기



#CustomerId 인코딩(원핫안되니까 라벨씀..타깃을 이럴때 써야하나싶음 쩔수임)
label_encoder = LabelEncoder()
df['CustomerId'] = label_encoder.fit_transform(df['CustomerId'])


#Surname 원핫인코딩한다음에 matrix로 저장
encoder = OneHotEncoder(sparse_output=True, drop='first')#false는 밀집행렬 true는 csr희소행렬이다. 밑에 보면 sparse.csr_matrix를 안한것을 볼 수 있다.
encoded_cat_matrix_1=encoder.fit_transform(df[['Surname']])
#Surname원핫인코딩해줬으니 Surname변수 df에서 없애버려야됨 df도 csr해줄려면 (object형은 없어야함 최소 숫자나 범주)
df = df.drop(['Surname'], axis=1)



#원래데이터와 Surname 데이터를 수평으로 저장하는데 format이 'csr'임
from scipy import sparse
all_data_sprs=sparse.hstack([sparse.csr_matrix(df),
                             encoded_cat_matrix_1],
                             format='csr')

all_data_sprs

<275057x2904 sparse matrix of type '<class 'numpy.float64'>'
	with 3072713 stored elements in Compressed Sparse Row format>

In [73]:
num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values

#평가지표 계산함수 작성
from sklearn.metrics import roc_auc_score


# ROC AUC 계산
#roc_auc = roc_auc_score(y_true, y_scores)
#y_true는 실제 값 y_scores는 예측 확률값이다.


#Lightgbm용 roc_auc 계산 함수
def roc_auc(preds,dtrain):
    labels=dtrain.get_label()
    return 'roc_auc', roc_auc_score(labels,preds), True



In [76]:
#베이지안 최적화 준비
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# 8:2 비율로 훈련 데이터, 검증 데이터 분리 (베이지안 최적화 수행용)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.2, 
                                                      random_state=0)

# 베이지안 최적화용 데이터셋
bayes_dtrain = lgb.Dataset(X_train, y_train)
bayes_dvalid = lgb.Dataset(X_valid, y_valid)

# 베이지안 최적화를 위한 하이퍼파라미터 범위
param_bounds = {'num_leaves': (30, 40),
                'lambda_l1': (0.7, 0.9),
                'lambda_l2': (0.9, 1),
                'feature_fraction': (0.6, 0.7),
                'bagging_fraction': (0.6, 0.9),
                'min_child_samples': (6, 10),
                'min_child_weight': (10, 40)}

# 값이 고정된 하이퍼파라미터
fixed_params = {'objective': 'binary',
                'learning_rate': 0.005,
                'bagging_freq': 1,
                'force_row_wise': True,
                'random_state': 1991}
def eval_function(num_leaves, lambda_l1, lambda_l2, feature_fraction,
                  bagging_fraction, min_child_samples, min_child_weight):
    '''최적화하려는 평가지표 계산 함수'''
    
    # 베이지안 최적화를 수행할 하이퍼파라미터 
    params = {'num_leaves': int(round(num_leaves)),
              'lambda_l1': lambda_l1,
              'lambda_l2': lambda_l2,
              'feature_fraction': feature_fraction,
              'bagging_fraction': bagging_fraction,
              'min_child_samples': int(round(min_child_samples)),
              'min_child_weight': min_child_weight,
              'feature_pre_filter': False}
    # 고정된 하이퍼파라미터도 추가
    params.update(fixed_params)
    
    print('하이퍼파라미터:', params)    
    
    # LightGBM 모델 훈련
    lgb_model = lgb.train(params=params, 
                           train_set=bayes_dtrain,
                           num_boost_round=2500,
                           valid_sets=bayes_dvalid,
                           feval=roc_auc,
                           early_stopping_rounds=300,
                           verbose_eval=False)
    # 검증 데이터로 예측 수행
    preds = lgb_model.predict(X_valid) 
    # rocauc 계산
    roc_score = roc_auc_score(y_valid, preds)
    print(f'roc 점수 : {roc_score}\n')
    
    return roc_score


In [77]:
#최적화 수행
from bayes_opt import BayesianOptimization

# 베이지안 최적화 객체 생성
optimizer = BayesianOptimization(f=eval_function,      # 평가지표 계산 함수
                                 pbounds=param_bounds, # 하이퍼파라미터 범위
                                 random_state=0)
# 베이지안 최적화 수행
optimizer.maximize(init_points=3, n_iter=6)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | min_ch... | min_ch... | num_le... |
-------------------------------------------------------------------------------------------------------------
하이퍼파라미터: {'num_leaves': 34, 'lambda_l1': 0.8205526752143287, 'lambda_l2': 0.9544883182996897, 'feature_fraction': 0.6715189366372419, 'bagging_fraction': 0.7646440511781974, 'min_child_samples': 8, 'min_child_weight': 29.376823391999682, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}


C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 28002, number of negative: 104025
[LightGBM] [Info] Total Bins 6612
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 2761
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.212093 -> initscore=-1.312355
[LightGBM] [Info] Start training from score -1.312355
roc 점수 : 0.8912956409912415

| 1         | 0.8913    | 0.7646    | 0.6715    | 0.8206    | 0.9545    | 7.695     | 29.38     | 34.38     |
하이퍼파라미터: {'num_leaves': 39, 'lambda_l1': 0.7766883037651555, 'lambda_l2': 0.9791725038082665, 'feature_fraction': 0.6963662760501029, 'bagging_fraction': 0.867531900234624, 'min_child_samples': 8, 'min_child_weight': 27.04133683281797, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}
[LightGBM] [Info] Number of positive: 28002, number of negative: 104025
[LightGBM] [Info] Total Bins 6612
[LightGBM] [Info] Number of data points in

C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


roc 점수 : 0.891971139938427

| 2         | 0.892     | 0.8675    | 0.6964    | 0.7767    | 0.9792    | 8.116     | 27.04     | 39.26     |
하이퍼파라미터: {'num_leaves': 40, 'lambda_l1': 0.7040436794880651, 'lambda_l2': 0.9832619845547939, 'feature_fraction': 0.608712929970154, 'bagging_fraction': 0.6213108174593661, 'min_child_samples': 9, 'min_child_weight': 36.10036444740457, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}
[LightGBM] [Info] Number of positive: 28002, number of negative: 104025
[LightGBM] [Info] Total Bins 6612
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 2761
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.212093 -> initscore=-1.312355
[LightGBM] [Info] Start training from score -1.312355


...

roc 점수 : 0.8932035829324644

| 7         | 0.8932    | 0.6       | 0.7       | 0.7       | 1.0       | 10.0      | 10.0      | 38.19     |
하이퍼파라미터: {'num_leaves': 33, 'lambda_l1': 0.7, 'lambda_l2': 0.9, 'feature_fraction': 0.7, 'bagging_fraction': 0.6, 'min_child_samples': 6, 'min_child_weight': 10.0, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}
[LightGBM] [Info] Number of positive: 28002, number of negative: 104025
[LightGBM] [Info] Total Bins 6612
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 2761
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.212093 -> initscore=-1.312355
[LightGBM] [Info] Start training from score -1.312355


C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


roc 점수 : 0.8932049347115099

| 8         | 0.8932    | 0.6       | 0.7       | 0.7       | 0.9       | 6.0       | 10.0      | 33.41     |
하이퍼파라미터: {'num_leaves': 34, 'lambda_l1': 0.7032881500564242, 'lambda_l2': 0.9950736372987603, 'feature_fraction': 0.6919553690043856, 'bagging_fraction': 0.6204662597322463, 'min_child_samples': 6, 'min_child_weight': 10.048543421515697, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}
[LightGBM] [Info] Number of positive: 28002, number of negative: 104025
[LightGBM] [Info] Total Bins 6612
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 2761


C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.212093 -> initscore=-1.312355
[LightGBM] [Info] Start training from score -1.312355
roc 점수 : 0.8932023585752144

| 9         | 0.8932    | 0.6205    | 0.692     | 0.7033    | 0.9951    | 6.014     | 10.05     | 33.55     |


In [78]:
#결과확인
# 평가함수 점수가 최대일 때 하이퍼파라미터
max_params = optimizer.max['params']
max_params

{'bagging_fraction': 0.6,
 'feature_fraction': 0.7,
 'lambda_l1': 0.7,
 'lambda_l2': 0.9,
 'min_child_samples': 6.0,
 'min_child_weight': 10.0,
 'num_leaves': 33.41386040721548}

In [79]:
# 정수형 하이퍼파라미터 변환
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))
# 값이 고정된 하이퍼파라미터 추가
max_params.update(fixed_params)
max_params

{'bagging_fraction': 0.6,
 'feature_fraction': 0.7,
 'lambda_l1': 0.7,
 'lambda_l2': 0.9,
 'min_child_samples': 6,
 'min_child_weight': 10.0,
 'num_leaves': 33,
 'objective': 'binary',
 'learning_rate': 0.005,
 'bagging_freq': 1,
 'force_row_wise': True,
 'random_state': 1991}

In [82]:
#모델훈련및 성능검증
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기 생성
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds = np.zeros(X.shape[0]) 
# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds = np.zeros(X_test.shape[0]) 


import lightgbm as lgb

# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구 출력
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정 
    X_train, y_train = X[train_idx], y[train_idx]  # 훈련용 데이터 (iloc 사용)
    X_valid, y_valid = X[valid_idx], y[valid_idx]  # 검증용 데이터 (iloc 사용)

    # LightGBM 전용 데이터셋 생성 
    dtrain = lgb.Dataset(X_train, y_train)  # LightGBM 전용 훈련 데이터셋
    dvalid = lgb.Dataset(X_valid, y_valid)  # LightGBM 전용 검증 데이터셋
    
    # LightGBM 모델 훈련 
    lgb_model = lgb.train(params=max_params,        # 훈련용 하이퍼파라미터
                          train_set=dtrain,     # 훈련 데이터셋
                          num_boost_round=1000, # 부스팅 반복 횟수
                          valid_sets=dvalid,    # 성능 평가용 검증 데이터셋
                          feval=roc_auc,        # 검증용 평가지표
                          early_stopping_rounds=100, # 조기종료 조건
                          verbose_eval=100)     # 100번째마다 점수 출력
    
    # 테스트 데이터를 활용해 OOF 예측(이게 진짜 결과임 5로나눠서 더해주잖아)
    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
    
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측(각 폴드마다)->5로 안나누어주는 이유는 평균값을 낼이유가 없잖아 폴드마다 결과를 보여
    #주기만 하면되는데
    oof_val_preds[valid_idx] = lgb_model.predict(X_valid)
    
    # 검증 데이터 예측 확률에 대한 auc넓이
    score = roc_auc_score(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} auc넓이 : {score}\n')

######################################## 폴드 1 / 폴드 5 ########################################


C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 27937, number of negative: 104090
[LightGBM] [Info] Total Bins 4513
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 1712
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211601 -> initscore=-1.315304
[LightGBM] [Info] Start training from score -1.315304
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.421762	valid_0's roc_auc: 0.888626
[200]	valid_0's binary_logloss: 0.375763	valid_0's roc_auc: 0.889236
[300]	valid_0's binary_logloss: 0.351166	valid_0's roc_auc: 0.8895
[400]	valid_0's binary_logloss: 0.337763	valid_0's roc_auc: 0.889812
[500]	valid_0's binary_logloss: 0.33036	valid_0's roc_auc: 0.890117
[600]	valid_0's binary_logloss: 0.325693	valid_0's roc_auc: 0.890417
[700]	valid_0's binary_logloss: 0.32269	valid_0's roc_auc: 0.890716
[800]	valid_0's binary_logloss: 0.32078	valid_0's roc_auc: 0.891126
[900]	valid_0's binary_logloss: 0.319294	valid_0's roc_

C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 27937, number of negative: 104090
[LightGBM] [Info] Total Bins 4522
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 1716
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211601 -> initscore=-1.315304
[LightGBM] [Info] Start training from score -1.315304
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.418836	valid_0's roc_auc: 0.887516
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[200]	valid_0's binary_logloss: 0.373698	valid_0's roc_auc: 0.887796
[300]	valid_0's binary_logloss: 0.350035	valid_0's roc_auc: 0.888326
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[400]	valid_0's binary_logloss: 0.337648	valid_0's roc_auc: 0.888731
[500]	valid_0's binary_logloss: 0.330145	valid_0's roc_auc: 0.889096
[600]	valid_0's binary_logloss: 0.325752	valid_0's roc_auc: 0.889402
[700]	valid_0's binary_logloss: 0.3229

...

[LightGBM] [Info] Number of positive: 27937, number of negative: 104090
[LightGBM] [Info] Total Bins 4536
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 1723
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211601 -> initscore=-1.315304
[LightGBM] [Info] Start training from score -1.315304
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.416092	valid_0's roc_auc: 0.88669
[200]	valid_0's binary_logloss: 0.3741	valid_0's roc_auc: 0.887362
[300]	valid_0's binary_logloss: 0.350861	valid_0's roc_auc: 0.887794
[400]	valid_0's binary_logloss: 0.338169	valid_0's roc_auc: 0.888056
[500]	valid_0's binary_logloss: 0.33102	valid_0's roc_auc: 0.888391
[600]	valid_0's binary_logloss: 0.326648	valid_0's roc_auc: 0.888745
[700]	valid_0's binary_logloss: 0.323813	valid_0's roc_auc: 0.889142
[800]	valid_0's binary_logloss: 0.321878	valid_0's roc_auc: 0.88955
[900]	valid_0's binary_logloss: 0.320378	valid_0's roc_

C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 27936, number of negative: 104091
[LightGBM] [Info] Total Bins 4547
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 1728
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211593 -> initscore=-1.315349
[LightGBM] [Info] Start training from score -1.315349
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.422308	valid_0's roc_auc: 0.88543
[200]	valid_0's binary_logloss: 0.378334	valid_0's roc_auc: 0.886023
[300]	valid_0's binary_logloss: 0.354546	valid_0's roc_auc: 0.886701
[400]	valid_0's binary_logloss: 0.340532	valid_0's roc_auc: 0.887113
[500]	valid_0's binary_logloss: 0.332411	valid_0's roc_auc: 0.887433
[600]	valid_0's binary_logloss: 0.327668	valid_0's roc_auc: 0.887726
[700]	valid_0's binary_logloss: 0.32489	valid_0's roc_auc: 0.888014
[800]	valid_0's binary_logloss: 0.323061	valid_0's roc_auc: 0.888437
[900]	valid_0's binary_logloss: 0.321659	valid_0's r

C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\ncc05\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 27937, number of negative: 104091
[LightGBM] [Info] Total Bins 4484
[LightGBM] [Info] Number of data points in the train set: 132028, number of used features: 1697
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211599 -> initscore=-1.315314
[LightGBM] [Info] Start training from score -1.315314
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.419928	valid_0's roc_auc: 0.886046
[200]	valid_0's binary_logloss: 0.376136	valid_0's roc_auc: 0.886524
[300]	valid_0's binary_logloss: 0.352452	valid_0's roc_auc: 0.886821
[400]	valid_0's binary_logloss: 0.339445	valid_0's roc_auc: 0.887192
[500]	valid_0's binary_logloss: 0.331429	valid_0's roc_auc: 0.887525
[600]	valid_0's binary_logloss: 0.326943	valid_0's roc_auc: 0.887765
[700]	valid_0's binary_logloss: 0.324118	valid_0's roc_auc: 0.888061
[800]	valid_0's binary_logloss: 0.322224	valid_0's roc_auc: 0.888423
[900]	valid_0's binary_logloss: 0.320827	valid_0's

In [83]:
print('test데이터에대한 roc넓이', roc_auc_score(y, oof_val_preds))

test데이터에대한 roc넓이 0.8904299404267852


In [84]:
submission['Exited']=oof_test_preds
submission.to_csv('submission.csv',index=False)